# mRAG-01

## Imports

In [25]:
import asyncio
import requests
import httpx

## Variables

In [5]:
# Define Ollama endpoints
ollama_base_url = "http://127.0.0.1:11434"
ollama_chat = "/api/chat"
ollama_embedding = "api/embed"

# Define Ollama models
embedding_model = "mxbai-embedding-large"
llm_model = "llama3.1:8b"

## Key Functions

In [6]:
async def get_llm_response(in_text, timeout=60.0):
    data = {
        "model": "llama3.1:8b",
        "messages": [
            {"role": "user", "content": in_text}
        ],
        "stream": False
    }
    
    try:
        # Create a client with an increased timeout
        async with httpx.AsyncClient(timeout=httpx.Timeout(timeout)) as client:
            response = await client.post('http://127.0.0.1:11434/api/chat', json=data)
            response.raise_for_status()
            
            result = response.json()
            
            # Extract the response text based on Ollama's API structure
            if "message" in result and "content" in result["message"]:
                return result["message"]["content"].strip()
            else:
                print('Unexpected response structure:', result)
                return None
                
    except httpx.ReadTimeout:
        print("Request timed out. The Ollama server might be busy or the model is taking too long to respond.")
        return None
    except httpx.ConnectError:
        print("Could not connect to the Ollama server. Make sure it's running at http://127.0.0.1:11434.")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [10]:
async def get_embeddings(text, model="mxbai-embed-large", timeout=60.0):
    """
    Get embeddings for the provided text using Ollama's API.
    
    Args:
        text: The text to generate embeddings for
        model: The model to use for embeddings (default: "llama3.1:8b")
        timeout: Timeout in seconds (default: 60 seconds)
    
    Returns:
        A list of embedding values or None if an error occurs
    """
    data = {
        "model": model,
        "prompt": text,
        "options": {
            "temperature": 0.0  # Lower temperature for more deterministic embeddings
        }
    }
    
    try:
        # Create a client with an increased timeout
        async with httpx.AsyncClient(timeout=httpx.Timeout(timeout)) as client:
            response = await client.post('http://127.0.0.1:11434/api/embeddings', json=data)
            response.raise_for_status()
            
            result = response.json()
            
            # Extract the embeddings from the response
            if "embedding" in result:
                return result["embedding"]
            else:
                print('No embeddings found in response:', result)
                return None
                
    except httpx.ReadTimeout:
        print("Request timed out. The Ollama server might be busy or the model is taking too long to respond.")
        return None
    except httpx.ConnectError:
        print("Could not connect to the Ollama server. Make sure it's running at http://127.0.0.1:11434.")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

## Testing

In [18]:
prompt = "This is a test prompt"

In [19]:
response = await get_llm_response(prompt)

In [20]:
print(response)

It looks like you've entered a test prompt. What would you like to do next? Would you like to:

A) Proceed with the actual test
B) Change or modify the prompt in some way
C) Cancel the test and start something new

Let me know your preference!


In [23]:
embeddings = await get_embeddings(prompt)

/tmp/ipykernel_43729/2521945630.py:1: RuntimeWarning: coroutine 'get_embeddings' was never awaited
  embeddings = await get_embeddings(prompt)


In [24]:
print(embeddings)

[0.5911464095115662, -0.11597968637943268, 0.05700969696044922, 0.26048168540000916, -0.9942144155502319, -0.29364389181137085, -0.25475233793258667, 0.13895362615585327, 0.09696785360574722, 1.1620640754699707, 0.01398104801774025, 0.3735347390174866, -0.18165525794029236, -0.17336203157901764, -0.3030553162097931, -0.6371006965637207, -0.3892456889152527, -0.17507749795913696, -0.5196360349655151, -0.9571660757064819, -0.27979445457458496, 0.4817340075969696, -0.8267162442207336, -0.24091395735740662, -0.5530280470848083, 0.11845100671052933, -0.19183573126792908, 0.47601795196533203, 0.49591314792633057, 0.701909065246582, -0.08509989082813263, 0.49282306432724, -0.11409945785999298, -0.6565920114517212, 0.11620348691940308, -0.5949080586433411, 1.1418606042861938, -0.3908167779445648, 0.05012013018131256, -0.37343892455101013, -0.1175934374332428, 0.16913989186286926, 0.4507707953453064, -0.6703962683677673, -1.4234727621078491, -0.782892107963562, -0.636871874332428, -0.4608845710

This is a test